In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 연결

Mounted at /content/drive


In [2]:
import pandas as pd
filename = '/content/drive/My Drive/data1_scale.csv'
dat = pd.read_csv(filename)
dat.head()

# 드라이브에서 파일 불러오기

,credit_score,desired_amount,existing_loan_amt,loan_limit,loan_rate,is_applied,age,mean_exloan
0,0.633333,0.509254,0.010117,0.0003,0.702703,0.0,0.342857,0.013018
1,0.633333,0.509254,0.010117,0.0001,0.994595,0.0,0.342857,0.013018
2,0.488889,0.428437,0.008520,0.0030,0.886486,1.0,0.385714,0.016444
3,0.677778,0.405797,0.003727,0.0009,0.427027,0.0,0.042857,0.007194
4,0.677778,0.405797,0.003727,0.0010,0.664865,0.0,0.042857,0.007194


In [1]:
import pandas as pd
dat = pd.read_csv('C:\\Users\\jihun\\Downloads\\data1_scale.csv')

In [3]:
dat.describe()

,credit_score,desired_amount,existing_loan_amt,loan_limit,loan_rate,is_applied,age,mean_exloan
count,8.014178e+06,8.014178e+06,8.014178e+06,8.014178e+06,8.014178e+06,8.014178e+06,8.014178e+06,8.014178e+06
mean,7.179291e-01,4.923586e-01,9.350715e-03,2.218897e-03,6.125051e-01,5.405595e-02,2.899108e-01,1.084055e-02
std,1.408846e-01,1.696780e-01,1.266045e-02,2.247384e-03,2.088818e-01,2.261281e-01,1.349975e-01,1.623827e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e-01,3.642888e-01,1.863685e-03,9.000000e-04,4.432432e-01,0.000000e+00,1.857143e-01,2.569373e-03
50%,7.000000e-01,5.092539e-01,5.324814e-03,1.700000e-03,6.378378e-01,0.000000e+00,2.857143e-01,6.509079e-03
75%,8.333333e-01,6.189157e-01,1.118211e-02,3.200000e-03,7.729730e-01,0.000000e+00,3.857143e-01,1.271840e-02
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [2]:
import numpy as np
import pandas as pd


dat['is_applied'] = dat['is_applied'].astype('category')


# 타겟 데이터 타입 변경

### train_test_split

In [3]:
from sklearn.model_selection import train_test_split
feature = dat.drop(columns='is_applied')
target = dat['is_applied']
X_train,X_test,y_train,y_test = train_test_split(feature,target,test_size=0.2,random_state=2,stratify=target)

# train, test 데이터로 분리.

In [6]:
X_train.to_csv("X_train.csv",index=False)
y_train.to_csv("y_train.csv",index=False)
X_test.to_csv("X_test.csv",index=False)
y_test.to_csv("y_test.csv",index=False)

# 기본 train, test 데이터 파일로 저장.

### 언더샘플링

In [7]:
from imblearn.under_sampling import NearMiss

nearmiss = NearMiss(version=1, n_neighbors=3)
X_train_under, y_train_under = nearmiss.fit_resample(X_train,y_train)
print('nearmiss 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('nearmiss 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_under.shape, y_train_under.shape)


print('\n nearmiss 적용 전 레이블 값 분포: \n', pd.Series(y_train).value_counts())
print('\n nearmiss 적용 후 레이블 값 분포: \n', pd.Series(y_train_under).value_counts())

# nearmiss version 1 under sampling  

nearmiss 적용 전 학습용 피처/레이블 데이터 세트:  (6010633, 7) (6010633,)
nearmiss 적용 후 학습용 피처/레이블 데이터 세트:  (649820, 7) (649820,)

 nearmiss 적용 전 레이블 값 분포: 
 0.0    5685723
1.0     324910
Name: is_applied, dtype: int64

 nearmiss 적용 후 레이블 값 분포: 
 0.0    324910
1.0    324910
Name: is_applied, dtype: int64


In [8]:
X_train_under.to_csv("X_train_under.csv",index=False)
y_train_under.to_csv("y_train_under.csv",index=False)

# under sampling 한 데이터들을 파일로 저장.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score,confusion_matrix, classification_report,roc_auc_score

model = LogisticRegression()
model.fit(X_train_under, y_train_under)
y_pred = model.predict(X_test)

print(confusion_matrix(y_pred,y_test))
print(f1_score(y_pred,y_test))
print(roc_auc_score(y_pred,y_test))

In [ ]:
from xgboost import XGBClassifier, plot_importance

model = XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
xgb_model = model.fit(X_train_smote, y_train_smote)

y_pred = xgb_model.predict(X_test)
y_pred_probs = xgb_model.predict_proba(X_test)[:,1]

print(confusion_matrix(y_pred,y_test))
print(f1_score(y_pred,y_test))
print(roc_auc_score(y_pred,y_test))

##  오버샘플링

### SMOTE 오버샘플링

In [9]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)


In [10]:
print('smote 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('smote 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_smote.shape, y_train_smote.shape)


print('\n smote 적용 전 레이블 값 분포: \n', pd.Series(y_train).value_counts())
print('\n smote 적용 후 레이블 값 분포: \n', pd.Series(y_train_smote).value_counts())

smote 적용 전 학습용 피처/레이블 데이터 세트:  (6010633, 7) (6010633,)
smote 적용 후 학습용 피처/레이블 데이터 세트:  (11371446, 7) (11371446,)

 smote 적용 전 레이블 값 분포: 
 0.0    5685723
1.0     324910
Name: is_applied, dtype: int64

 smote 적용 후 레이블 값 분포: 
 0.0    5685723
1.0    5685723
Name: is_applied, dtype: int64


In [11]:
X_train_smote.to_csv("X_train_smote.csv",index=False)
y_train_smote.to_csv("y_train_smote.csv",index=False)

### BorderlineSMOTE 오버샘플링

In [12]:
from imblearn.over_sampling import BorderlineSMOTE
bsmote = BorderlineSMOTE(random_state = 101, kind = 'borderline-1')
X_train_border, y_train_border = bsmote.fit_resample(X_train, y_train)

In [13]:
print('borderline smote 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('borderline smote 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_border.shape, y_train_border.shape)


print('\n borderline smote 적용 전 레이블 값 분포: \n', pd.Series(y_train).value_counts())
print('\n borderline smote 적용 후 레이블 값 분포: \n', pd.Series(y_train_border).value_counts())

borderline smote 적용 전 학습용 피처/레이블 데이터 세트:  (6010633, 7) (6010633,)
borderline smote 적용 후 학습용 피처/레이블 데이터 세트:  (11371446, 7) (11371446,)

 borderline smote 적용 전 레이블 값 분포: 
 0.0    5685723
1.0     324910
Name: is_applied, dtype: int64

 borderline smote 적용 후 레이블 값 분포: 
 0.0    5685723
1.0    5685723
Name: is_applied, dtype: int64


In [14]:
X_train_border.to_csv("X_train_border.csv",index=False)
y_train_border.to_csv("y_train_border.csv",index=False)

###  ADASYN 오버샘플링

In [15]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=22)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

In [16]:
print('adasyn 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('adasyn 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_adasyn.shape, y_train_adasyn.shape)


print('\n adasyn 적용 전 레이블 값 분포: \n', pd.Series(y_train).value_counts())
print('\n adasyn 적용 후 레이블 값 분포: \n', pd.Series(y_train_adasyn).value_counts())

adasyn 적용 전 학습용 피처/레이블 데이터 세트:  (6010633, 7) (6010633,)
adasyn 적용 후 학습용 피처/레이블 데이터 세트:  (11431639, 7) (11431639,)

 adasyn 적용 전 레이블 값 분포: 
 0.0    5685723
1.0     324910
Name: is_applied, dtype: int64

 adasyn 적용 후 레이블 값 분포: 
 1.0    5745916
0.0    5685723
Name: is_applied, dtype: int64


In [17]:
X_train_adasyn.to_csv("X_train_adasyn.csv",index=False)
y_train_adasyn.to_csv("y_train_adasyn.csv",index=False)

### SMOTETOMEK 오버샘플링

In [18]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_train_smtomek, y_train_smtomek = smoteto.fit_resample(X_train, y_train)

In [19]:
print('smotetomek 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('smotetomek 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_smtomek.shape, y_train_smtomek.shape)


print('\n smotetomek 적용 전 레이블 값 분포: \n', pd.Series(y_train).value_counts())
print('\n smotetomek 적용 후 레이블 값 분포: \n', pd.Series(y_train_smtomek).value_counts())

smotetomek 적용 전 학습용 피처/레이블 데이터 세트:  (6010633, 7) (6010633,)
smotetomek 적용 후 학습용 피처/레이블 데이터 세트:  (11345721, 7) (11345721,)

 smotetomek 적용 전 레이블 값 분포: 
 0.0    5685723
1.0     324910
Name: is_applied, dtype: int64

 smotetomek 적용 후 레이블 값 분포: 
 1.0    5685723
0.0    5659998
Name: is_applied, dtype: int64


In [20]:
X_train_smtomek.to_csv("X_train_smtomek.csv",index=False)
y_train_smtomek.to_csv("y_train_smtomek.csv",index=False)

## 분류모델 (GridSearchCV전, 기본 모델)

### 의사결정나무

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96   1916789
         1.0       0.20      0.21      0.20     99255

    accuracy                           0.92   2016044
   macro avg       0.58      0.58      0.58   2016044
weighted avg       0.92      0.92      0.92   2016044



0.20478340956200972

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train_under,y_train_under)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.14      0.92      0.24    293780
         1.0       0.77      0.05      0.09   1722264

    accuracy                           0.17   2016044
   macro avg       0.46      0.48      0.17   2016044
weighted avg       0.68      0.17      0.11   2016044



0.09147998619285436

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train_smote,y_train_smote)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)


              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96   1898392
         1.0       0.26      0.24      0.25    117652

    accuracy                           0.92   2016044
   macro avg       0.61      0.60      0.60   2016044
weighted avg       0.91      0.92      0.91   2016044



0.2537424223679327

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train_border,y_train_border)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96   1898292
         1.0       0.27      0.25      0.26    117752

    accuracy                           0.92   2016044
   macro avg       0.61      0.60      0.61   2016044
weighted avg       0.91      0.92      0.91   2016044



0.25548528424548633

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train_adasyn,y_train_adasyn)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96   1898429
         1.0       0.27      0.25      0.25    117615

    accuracy                           0.92   2016044
   macro avg       0.61      0.60      0.61   2016044
weighted avg       0.91      0.92      0.91   2016044



0.2547030982778507

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train_smtomek,y_train_smtomek)
y_pred = dtree.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.95      0.96      0.95   1893909
         1.0       0.27      0.24      0.26    122135

    accuracy                           0.91   2016044
   macro avg       0.61      0.60      0.60   2016044
weighted avg       0.91      0.91      0.91   2016044



0.25516557123484124

### KNN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97   1986352
         1.0       0.09      0.33      0.14     29692

    accuracy                           0.94   2016044
   macro avg       0.54      0.64      0.56   2016044
weighted avg       0.98      0.94      0.96   2016044



0.14214680119106127

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_over,y_train_over)
y_pred = knn.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.87      0.97      0.91   1709118
         1.0       0.49      0.18      0.26    306926

    accuracy                           0.85   2016044
   macro avg       0.68      0.57      0.59   2016044
weighted avg       0.81      0.85      0.81   2016044



0.25862973354058494

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_border,y_train_border)
y_pred = knn.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93   1756381
         1.0       0.45      0.19      0.27    259663

    accuracy                           0.87   2016044
   macro avg       0.67      0.58      0.60   2016044
weighted avg       0.83      0.87      0.84   2016044



0.2675173415504908

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_adasyn,y_train_adasyn)
y_pred = knn.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.86      0.97      0.91   1699068
         1.0       0.50      0.17      0.26    316976

    accuracy                           0.84   2016044
   macro avg       0.68      0.57      0.58   2016044
weighted avg       0.81      0.84      0.81   2016044



0.25634326955606507

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train_smtomek,y_train_smtomek)
y_pred = knn.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

              precision    recall  f1-score   support

         0.0       0.86      0.97      0.91   1703798
         1.0       0.50      0.17      0.26    312246

    accuracy                           0.85   2016044
   macro avg       0.68      0.57      0.59   2016044
weighted avg       0.81      0.85      0.81   2016044



0.25764163091148395

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

print(classification_report(y_pred,y_test))
f1_score(y_pred,y_test)

### 랜덤포레스트

In [21]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train,y_train)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_report = classification_report(y_pred,y_test)
rf_cf = confusion_matrix(y_pred,y_test)
rf_f1 = f1_score(y_pred,y_test)
print(rf_report)
print(rf_cf)
print(rf_f1)

              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97   1963075
         1.0       0.16      0.42      0.23     40470

    accuracy                           0.94   2003545
   macro avg       0.57      0.69      0.60   2003545
weighted avg       0.97      0.94      0.96   2003545

[[1871760   91315]
 [  23481   16989]]
0.2283866804683614


In [22]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train_under,y_train_under)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_under_report = classification_report(y_pred,y_test)
rf_under_cf = confusion_matrix(y_pred,y_test)
rf_under_f1 = f1_score(y_pred,y_test)
print(rf_under_report)
print(rf_under_cf)
print(rf_under_f1)

              precision    recall  f1-score   support

         0.0       0.18      0.93      0.30    357340
         1.0       0.78      0.05      0.10   1646205

    accuracy                           0.21   2003545
   macro avg       0.48      0.49      0.20   2003545
weighted avg       0.67      0.21      0.13   2003545

[[ 333308   24032]
 [1561933   84272]]
0.09606334307774995


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train_smote,y_train_smote)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_smote_report = classification_report(y_pred,y_test)
rf_smote_cf = confusion_matrix(y_pred,y_test)
rf_smote_f1 = f1_score(y_pred,y_test)
print(rf_smote_report)
print(rf_smote_cf)
print(rf_smote_f1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train_border,y_train_border)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_border_report = classification_report(y_pred,y_test)
rf_border_cf = confusion_matrix(y_pred,y_test)
rf_border_f1 = f1_score(y_pred,y_test)
print(rf_border_report)
print(rf_border_cf)
print(rf_border_f1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train_adasyn,y_train_adasyn)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_adasyn_report = classification_report(y_pred,y_test)
rf_adasyn_cf = confusion_matrix(y_pred,y_test)
rf_adasyn_f1 = f1_score(y_pred,y_test)
print(rf_adasyn_report)
print(rf_adasyn_cf)
print(rf_adasyn_f1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier()
rforest.fit(X_train_smtomek,y_train_smtomek)
y_pred = rforest.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report, f1_score

rf_smtomek_report = classification_report(y_pred,y_test)
rf_smtomek_cf = confusion_matrix(y_pred,y_test)
rf_smtomek_f1 = f1_score(y_pred,y_test)
print(rf_smtomek_report)
print(rf_smtomek_cf)
print(rf_smtomek_f1)

### 로지스틱 회귀, XGBoost, Adaboost

## GridSearchCV

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(),params,scoring='f1')
grid_search.fit(X_train,y_train)
print('그리드 서치 결과 가장 높은 f1 score : {}'.format(grid_search.best_score_))
print('그리드 서치 결과 가장 좋은 초매개변수 : {}'.format(grid_search.best_params_))
knn_grid_score = grid_search.score(X_test,y_test)
print('테스트 데이터 f1 score : {}'.format(grid_search.score(X_test,y_test)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {
    'n_neighbors' : list(range(1,11)),
    'weights' : ["uniform", "distance"]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(),params,scoring='f1')
grid_search.fit(X_train_border,y_train_border)
print('그리드 서치 결과 가장 높은 f1 score : {}'.format(grid_search.best_score_))
print('그리드 서치 결과 가장 좋은 초매개변수 : {}'.format(grid_search.best_params_))
knn_grid_score = grid_search.score(X_test,y_test)
print('테스트 데이터 f1 score : {}'.format(grid_search.score(X_test,y_test)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feature = dat.drop(columns='is_applied')
target = dat['is_applied']
X_train,X_test,y_train,y_test = train_test_split(feature,target,shuffle=True,random_state=2,stratify=target)


from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)

from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100,200,300],
          'max_depth' : [10,20,30],
          'max_features': [0.1,0.3,0.5,0.7],
          'min_samples_split' : [2,5,10]}
grid_search = GridSearchCV(RandomForestClassifier(),params,scoring='f1',cv=skf)
grid_search.fit(X_train,y_train)

In [ ]:
print('그리드 서치 결과 가장 높은 f1 score : {}'.format(grid_search.best_score_))
print('그리드 서치 결과 가장 좋은 초매개변수 : {}'.format(grid_search.best_params_))
rf_grid_score = grid_search.score(X_test,y_test)
print('테스트 데이터 f1 score : {}'.format(grid_search.score(X_test,y_test)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(),params,scoring='f1',cv=skf)
grid_search.fit(X_train,y_train)
print('그리드 서치 결과 가장 높은 f1 score : {}'.format(grid_search.best_score_))
print('그리드 서치 결과 가장 좋은 초매개변수 : {}'.format(grid_search.best_params_))
knn_grid_score = grid_search.score(X_test,y_test)
print('테스트 데이터 f1 score : {}'.format(grid_search.score(X_test,y_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

grid_search = GridSearchCV(LogisticRegression(),params,scoring='f1')
grid_search.fit(X_train_border,y_train_border)
print('그리드 서치 결과 가장 높은 f1 score : {}'.format(grid_search.best_score_))
print('그리드 서치 결과 가장 좋은 초매개변수 : {}'.format(grid_search.best_params_))
lr_grid_score = grid_search.score(X_test,y_test)
print('테스트 데이터 f1 score : {}'.format(grid_search.score(X_test,y_test)))

## 스태킹 앙상블

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=0)
ada = AdaBoostClassifier(n_estimators=100)
xgb = XGBClassifier()

# 위 모델들의 파라미터들은 GridSearchCV 이후 나온 파라미터들을 적용할 예정.


def get_stacking_data(model, X_train, y_train, X_test, n_folds = 5 ):
    stk = StratifiedKFold(n_split=n_folds,random_state=2)

    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train.shape[0] ,1 ))
    test_pred = np.zeros((X_test.shape[0],n_folds))

    for cnt , (train_index, valid_index) in enumerate(skf.split(X_train,y_train)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        X_tr = X_train[train_index] 
        y_tr = y_train[train_index] 
        X_te = X_train[valid_index]  

        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, cnt] = model.predict(X_test)
   
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    

    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean


rf_train, rf_test = get_stacking_data(rf, X_train, y_train, X_test, 5)
xgb_train, xgb_test = get_stacking_data(xgb, X_train, y_train, X_test,  5)    
ada_train, ada_test = get_stacking_data(ada, X_train, y_train, X_test, 5)

Stack_final_X_train = np.concatenate((rf_train, xgb_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((rf_test, xgb_test, ada_test), axis=1)
print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

# 스태킹 앙상블

In [ ]:
lgbm.fit(Stack_final_X_train,y_train)
stack_pred = lgbm.predict(Stack_final_X_test)
print(classification_report(stack_pred,y_test))
print(confusion_matrix(stack_pred,y_test))
print(f1_score(stack_pred,y_test))
print(roc_auc_score(stack_pred,y_test))

# 스태킹 후 나온 최종 데이터들을 이용하여 LGBM 모델로 진행 예정.

In [ ]:
f1 = {'knn':knnscore,'rf':rfscore,'lr':lrscore,'knn_grid':knn_grid_score,'rf_grid':rf_grid_score,'lr':lr_grid_score}
import matplotlib.pyplot as plt


sort_dic = dict(sorted(f1.items(), key = lambda pair : pair[1], reverse = True ))
print(sort_dic)
names = list(sort_dic.keys())
values = list(sort_dic.values())

plt.barh(range(len(sort_dic)), values, tick_label=names)
plt.show()

In [8]:
list1 = ['a','b','c']
dict1 = {'a':123,'b':234}
list2 = {'d','e','f'}

In [5]:
print(list1,dict1)

['a', 'b', 'c'] {'a': 123, 'b': 234}


In [12]:
import pickle
 
name = 'james'
age = 17
address = '서울시 서초구 반포동'
scores = {'korean': 90, 'english': 95, 'mathematics': 85, 'science': 82}
 
with open('james.p', 'wb') as file:    # james.p 파일을 바이너리 쓰기 모드(wb)로 열기
    pickle.dump(name, file)
    pickle.dump(age, file)
    pickle.dump(address, file)
    pickle.dump(scores, file)

In [13]:
import pickle
 
with open('james.p', 'rb') as file:    # james.p 파일을 바이너리 읽기 모드(rb)로 열기
    name = pickle.load(file)
    age = pickle.load(file)
    address = pickle.load(file)
    scores = pickle.load(file)
    print(name)
    print(age)
    print(address)
    print(scores)

james
17
서울시 서초구 반포동
{'korean': 90, 'english': 95, 'mathematics': 85, 'science': 82}


In [ ]:
import pickle

with open